In [1]:
import itertools
from sqlalchemy import create_engine
import os
import json
from urllib.parse import urlparse
import numpy as np
import pandas as pd
import feedparser
from newsie.constants import FEED_URL
from newsie.utils import extract_html, get_summary, extract_links
import ssl
import duckdb

import networkx as nx
import matplotlib.pyplot as plt
import itertools

if hasattr(ssl, '_create_unverified_context'):
    ssl._create_default_https_context = ssl._create_unverified_context
%load_ext dotenv
%dotenv

In [2]:

feed = feedparser.parse(FEED_URL)
feed


{'bozo': False,
 'entries': [{'title': 'What former BYU forward Caleb Lohner said about playing his old team',
   'title_detail': {'type': 'text/plain',
    'language': None,
    'base': 'https://www.sltrib.com/rss/feed/?sec=/News/Utah/&level=1',
    'value': 'What former BYU forward Caleb Lohner said about playing his old team'},
   'links': [{'rel': 'alternate',
     'type': 'text/html',
     'href': 'https://www.sltrib.com/sports/byu-cougars/2024/01/08/what-former-byu-forward-caleb/'}],
   'link': 'https://www.sltrib.com/sports/byu-cougars/2024/01/08/what-former-byu-forward-caleb/',
   'id': 'https://www.sltrib.com/sports/byu-cougars/2024/01/08/what-former-byu-forward-caleb/',
   'guidislink': False,
   'authors': [{'name': 'Alex Vejar'}],
   'author': 'Alex Vejar',
   'author_detail': {'name': 'Alex Vejar'},
   'published': 'Mon, 08 Jan 2024 22:09:47 +0000',
   'published_parsed': time.struct_time(tm_year=2024, tm_mon=1, tm_mday=8, tm_hour=22, tm_min=9, tm_sec=47, tm_wday=0, tm_yda

In [3]:
df = pd.DataFrame(feed.entries)
df

,title,title_detail,links,link,id,guidislink,authors,author,author_detail,published,published_parsed,summary,summary_detail,content
0,What former BYU forward Caleb Lohner said abou...,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.sltrib.com/sports/byu-cougars/2024...,https://www.sltrib.com/sports/byu-cougars/2024...,False,[{'name': 'Alex Vejar'}],Alex Vejar,{'name': 'Alex Vejar'},"Mon, 08 Jan 2024 22:09:47 +0000","(2024, 1, 8, 22, 9, 47, 0, 8, 0)",Tuesday's game between BYU and Baylor will be ...,"{'type': 'text/html', 'language': None, 'base'...","[{'type': 'text/html', 'language': None, 'base..."
1,Bagley Cartoon: Man of La MAGA,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.sltrib.com/opinion/bagley/2024/01/...,https://www.sltrib.com/opinion/bagley/2024/01/...,False,[{'name': 'Pat Bagley'}],Pat Bagley,{'name': 'Pat Bagley'},"Mon, 08 Jan 2024 20:24:26 +0000","(2024, 1, 8, 20, 24, 26, 0, 8, 0)",,"{'type': 'text/html', 'language': None, 'base'...","[{'type': 'text/html', 'language': None, 'base..."
2,A new ‘frugal’ Republican is running to replac...,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.sltrib.com/news/politics/2024/01/0...,https://www.sltrib.com/news/politics/2024/01/0...,False,[{'name': 'Vanessa Hudson'}],Vanessa Hudson,{'name': 'Vanessa Hudson'},"Mon, 08 Jan 2024 22:59:44 +0000","(2024, 1, 8, 22, 59, 44, 0, 8, 0)",John Dougall says his years of being a watchdo...,"{'type': 'text/html', 'language': None, 'base'...","[{'type': 'text/html', 'language': None, 'base..."
3,Provo woman arrested after shooting husband sa...,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.sltrib.com/news/2024/01/08/provo-w...,https://www.sltrib.com/news/2024/01/08/provo-w...,False,[{'name': 'Scott D. Pierce'}],Scott D. Pierce,{'name': 'Scott D. Pierce'},"Mon, 08 Jan 2024 19:53:27 +0000","(2024, 1, 8, 19, 53, 27, 0, 8, 0)",A Provo woman who told police she shot and kil...,"{'type': 'text/html', 'language': None, 'base'...","[{'type': 'text/html', 'language': None, 'base..."
4,Lo que sabemos hasta ahora sobre los proyectos...,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.sltrib.com/news/politics/2024/01/0...,https://www.sltrib.com/news/politics/2024/01/0...,False,[{'name': 'Courtney Tanner'}],Courtney Tanner,{'name': 'Courtney Tanner'},"Mon, 08 Jan 2024 19:06:46 +0000","(2024, 1, 8, 19, 6, 46, 0, 8, 0)",Esto es lo que sabemos hasta ahora sobre los t...,"{'type': 'text/html', 'language': None, 'base'...","[{'type': 'text/html', 'language': None, 'base..."
5,"World-renowned ski mountaineer, environmental ...","{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.sltrib.com/news/politics/2024/01/0...,https://www.sltrib.com/news/politics/2024/01/0...,False,[{'name': 'Emily Anderson Stern'}],Emily Anderson Stern,{'name': 'Emily Anderson Stern'},"Mon, 08 Jan 2024 21:33:29 +0000","(2024, 1, 8, 21, 33, 29, 0, 8, 0)","First, she summited Mount Everest. Now, climat...","{'type': 'text/html', 'language': None, 'base'...","[{'type': 'text/html', 'language': None, 'base..."
6,Utah State’s surprise season continues with a ...,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.sltrib.com/sports/byu-cougars/2024...,https://www.sltrib.com/sports/byu-cougars/2024...,False,[{'name': 'Aaron Falk'}],Aaron Falk,{'name': 'Aaron Falk'},"Mon, 08 Jan 2024 18:38:51 +0000","(2024, 1, 8, 18, 38, 51, 0, 8, 0)",AP Top 25 basketball polls: Utah State makes i...,"{'type': 'text/html', 'language': None, 'base'...","[{'type': 'text/html', 'language': None, 'base..."
7,New Heber City police chief faces two lawsuit

In [4]:
df = pd.json_normalize(feed.entries)
df = df.fillna('')
df

,title,links,link,id,guidislink,authors,author,published,published_parsed,summary,content,title_detail.type,title_detail.language,title_detail.base,title_detail.value,author_detail.name,summary_detail.type,summary_detail.language,summary_detail.base,summary_detail.value
0,What former BYU forward Caleb Lohner said abou...,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.sltrib.com/sports/byu-cougars/2024...,https://www.sltrib.com/sports/byu-cougars/2024...,False,[{'name': 'Alex Vejar'}],Alex Vejar,"Mon, 08 Jan 2024 22:09:47 +0000","(2024, 1, 8, 22, 9, 47, 0, 8, 0)",Tuesday's game between BYU and Baylor will be ...,"[{'type': 'text/html', 'language': None, 'base...",text/plain,,https://www.sltrib.com/rss/feed/?sec=/News/Uta...,What former BYU forward Caleb Lohner said abou...,Alex Vejar,text/html,,https://www.sltrib.com/rss/feed/?sec=/News/Uta...,Tuesday's game between BYU and Baylor will be ...
1,Bagley Cartoon: Man of La MAGA,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.sltrib.com/opinion/bagley/2024/01/...,https://www.sltrib.com/opinion/bagley/2024/01/...,False,[{'name': 'Pat Bagley'}],Pat Bagley,"Mon, 08 Jan 2024 20:24:26 +0000","(2024, 1, 8, 20, 24, 26, 0, 8, 0)",,"[{'type': 'text/html', 'language': None, 'base...",text/plain,,https://www.sltrib.com/rss/feed/?sec=/News/Uta...,Bagley Cartoon: Man of La MAGA,Pat Bagley,text/html,,https://www.sltrib.com/rss/feed/?sec=/News/Uta...,
2,A new ‘frugal’ Republican is running to replac...,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.sltrib.com/news/politics/2024/01/0...,https://www.sltrib.com/news/politics/2024/01/0...,False,[{'name': 'Vanessa Hudson'}],Vanessa Hudson,"Mon, 08 Jan 2024 22:59:44 +0000","(2024, 1, 8, 22, 59, 44, 0, 8, 0)",John Dougall says his years of being a watchdo...,"[{'type': 'text/html', 'language': None, 'base...",text/plain,,https://www.sltrib.com/rss/feed/?sec=/News/Uta...,A new ‘frugal’ Republican is running to replac...,Vanessa Hudson,text/html,,https://www.sltrib.com/rss/feed/?sec=/News/Uta...,John Dougall says his years of being a watchdo...
3,Provo woman arrested after shooting husband sa...,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.sltrib.com/news/2024/01/08/provo-w...,https://www.sltrib.com/news/2024/01/08/provo-w...,False,[{'name': 'Scott D. Pierce'}],Scott D. Pierce,"Mon, 08 Jan 2024 19:53:27 +0000","(2024, 1, 8, 19, 53, 27, 0, 8, 0)",A Provo woman who told police she shot and kil...,"[{'type': 'text/html', 'language': None, 'base...",text/plain,,https://www.sltrib.com/rss/feed/?sec=/News/Uta...,Provo woman arrested after shooting husband sa...,Scott D. Pierce,text/html,,https://www.sltrib.com/rss/feed/?sec=/News/Uta...,A Provo woman who told police she shot and kil...
4,Lo que sabemos hasta ahora sobre los proyectos...,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.sltrib.com/news/politics/2024/01/0...,https://www.sltrib.com/news/politics/2024/01/0...,False,[{'name': 'Courtney Tanner'}],Courtney Tanner,"Mon, 08 Jan 2024 19:06:46 +0000","(2024, 1, 8, 19, 6, 46, 0, 8, 0)",Esto es lo que sabemos hasta ahora sobre los t...,"[{'type': 'text/html', 'language': None, 'base...",text/plain,,https://www.sltrib.com/rss/feed/?sec=/News/Uta...,Lo que sabemos hasta ahora sobre los proyectos...,Courtney Tanner,text/html,,https://www.sltrib.com/rss/feed/?sec=/News/Uta...,Esto es lo que sabemos hasta ahora sobre los t...
5,"World-renowned ski mountaineer, environmental ...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.sltrib.com/news/politics/2024/01/0...,https://www.sltrib.com/news/politics/2024/01/0...,False,[{'name': 'Emily Anderson Stern'}],Emily Anderson Stern,"Mon, 08 Jan 2024 21:33:29 +0000","(2024, 1, 8, 21, 33, 29, 0, 8, 0)","First, she summited Mount Everest. Now, climat...","[{'type': 'text/html', 'language': None, 'base...",text/plain,,https://www.sltrib.com/rss/feed/?sec=/News/Uta...,"World-renowned ski mountaineer, environmental ...",Emily And

In [5]:
df = df.convert_dtypes()
df.set_index('link')
df = df.drop(columns=['links', 'authors', 'published_parsed'])
# assume df is your DataFrame and 'content' is the column with list of dicts
df['content'] = df['content'].apply(lambda x: x[0]['value'] if x else '').astype(str)
df.dtypes

title                      string[python]
link                       string[python]
id                         string[python]
guidislink                        boolean
author                     string[python]
published                  string[python]
summary                    string[python]
content                            object
title_detail.type          string[python]
title_detail.language      string[python]
title_detail.base          string[python]
title_detail.value         string[python]
author_detail.name         string[python]
summary_detail.type        string[python]
summary_detail.language    string[python]
summary_detail.base        string[python]
summary_detail.value       string[python]
dtype: object

In [6]:
df['link_texts'], df['link_hrefs'] = zip(*df['content'].apply(extract_links))

In [7]:
df

,title,link,id,guidislink,author,published,summary,content,title_detail.type,title_detail.language,title_detail.base,title_detail.value,author_detail.name,summary_detail.type,summary_detail.language,summary_detail.base,summary_detail.value,link_texts,link_hrefs
0,What former BYU forward Caleb Lohner said abou...,https://www.sltrib.com/sports/byu-cougars/2024...,https://www.sltrib.com/sports/byu-cougars/2024...,False,Alex Vejar,"Mon, 08 Jan 2024 22:09:47 +0000",Tuesday's game between BYU and Baylor will be ...,"<p>It’s been nearly two years since<a href=""ht...",text/plain,,https://www.sltrib.com/rss/feed/?sec=/News/Uta...,What former BYU forward Caleb Lohner said abou...,Alex Vejar,text/html,,https://www.sltrib.com/rss/feed/?sec=/News/Uta...,Tuesday's game between BYU and Baylor will be ...,"[ Caleb Lohner transferred from BYU, career-hi...",[https://www.sltrib.com/sports/byu-cougars/202...
1,Bagley Cartoon: Man of La MAGA,https://www.sltrib.com/opinion/bagley/2024/01/...,https://www.sltrib.com/opinion/bagley/2024/01/...,False,Pat Bagley,"Mon, 08 Jan 2024 20:24:26 +0000",,<p><br /></p>,text/plain,,https://www.sltrib.com/rss/feed/?sec=/News/Uta...,Bagley Cartoon: Man of La MAGA,Pat Bagley,text/html,,https://www.sltrib.com/rss/feed/?sec=/News/Uta...,,[],[]
2,A new ‘frugal’ Republican is running to replac...,https://www.sltrib.com/news/politics/2024/01/0...,https://www.sltrib.com/news/politics/2024/01/0...,False,Vanessa Hudson,"Mon, 08 Jan 2024 22:59:44 +0000",John Dougall says his years of being a watchdo...,<p>John Dougall says his years of being a watc...,text/plain,,https://www.sltrib.com/rss/feed/?sec=/News/Uta...,A new ‘frugal’ Republican is running to replac...,Vanessa Hudson,text/html,,https://www.sltrib.com/rss/feed/?sec=/News/Uta...,John Dougall says his years of being a watchdo...,[Curtis launched his campaign to replace Mitt ...,[https://www.sltrib.com/news/politics/2024/01/...
3,Provo woman arrested after shooting husband sa...,https://www.sltrib.com/news/2024/01/08/provo-w...,https://www.sltrib.com/news/2024/01/08/provo-w...,False,Scott D. Pierce,"Mon, 08 Jan 2024 19:53:27 +0000",A Provo woman who told police she shot and kil...,<p>A Provo woman who told police she shot and ...,text/plain,,https://www.sltrib.com/rss/feed/?sec=/News/Uta...,Provo woman arrested after shooting husband sa...,Scott D. Pierce,text/html,,https://www.sltrib.com/rss/feed/?sec=/News/Uta...,A Provo woman who told police she shot and kil...,[],[]
4,Lo que sabemos hasta ahora sobre los proyectos...,https://www.sltrib.com/news/politics/2024/01/0...,https://www.sltrib.com/news/politics/2024/01/0...,False,Courtney Tanner,"Mon, 08 Jan 2024 19:06:46 +0000",Esto es lo que sabemos hasta ahora sobre los t...,"<p><i>To read this story in English, </i><a hr...",text/plain,,https://www.sltrib.com/rss/feed/?sec=/News/Uta...,Lo que sabemos hasta ahora sobre los proyectos...,Courtney Tanner,text/html,,https://www.sltrib.com/rss/feed/?sec=/News/Uta...,Esto es lo que sabemos hasta ahora sobre los t...,"[click here., fracasó aquí el año pasado, así ...",[https://www.sltrib.com/news/education/2024/01...
5,"World-renowned ski mountaineer, environmental ...",https://www.sltrib.com/news/politics/2024/01/0...,https://www.sltrib.com/news/politics/2024/01/0...,False,Emily Anderson Stern,"Mon, 08 Jan 2024 21:33:29 +0000","First, she summited Mount Everest. Now, climat...","<p>First, she <a href=""https://www.sltrib.com/...",text/plain,,https://www.sltrib.com/rss/feed/?sec=/News/Uta...,"World-renowned ski mountaineer, environmental ...",Emily Anderson Stern,text/html,,https://www.sltrib.com/rss/feed/?sec=/News/Uta...,"First, she summited Mount Everest. Now, climat...","[summited Mount Everest, #utpol, #Utah, #USSen...",[https://www.sltrib.com/news/2019/05/28/utah-m...
6,Utah State’s surprise season continues with a ...,https://www.sltrib.com/sports/byu-cougars/2024...,https://www.sltrib.com/sports/byu-cougars/2024...,False,Aaron Falk,"Mon, 08 Jan 2024 18:38:51 +0000",AP 

In [8]:
DB_NAME = "scratch_analysis_inline.duckdb"
con = duckdb.connect(DB_NAME, read_only=False)
con.register("scratch_table", df)
con.execute("SELECT * FROM scratch_table").fetchall()
con.close()

In [9]:
url = os.environ['PG_DB_URL']

# connection = psycopg.connect(url)
from sqlalchemy import create_engine
engine = create_engine(url)

In [10]:
# df_flat = pd.json_normalize(df.to_dict(orient='list'))
df.to_sql("feed_df", engine, if_exists='replace')

53

# DB Load Bookmark

In [ ]:
# 
# con = duckdb.connect(DB_NAME, read_only=False)
# df = con.execute("SELECT * FROM scratch_table").fetch_df()
# df

In [14]:
DB_NAME = "notebook.duckdb"
con = duckdb.connect(DB_NAME, read_only=False)
con.register("scratch_table", df)
df = con.execute("SELECT * FROM public.summarize_articles").fetch_df()

df.dtypes


id                  object
link_texts          object
link_hrefs          object
title               object
title_detail        object
links               object
link                object
guidislink            bool
authors             object
author              object
author_detail       object
published           object
published_parsed    object
summary             object
summary_detail      object
content             object
dtype: object

In [16]:
df = df.convert_dtypes()
df.dtypes

id                  string[python]
link_texts                  object
link_hrefs                  object
title               string[python]
title_detail                object
links                       object
link                string[python]
guidislink                 boolean
authors                     object
author              string[python]
author_detail               object
published           string[python]
published_parsed            object
summary             string[python]
summary_detail              object
content             string[python]
dtype: object

In [19]:
# Convert dictionary-like columns to JSON strings
dict_columns = ['link_texts', 'link_hrefs', 'title_detail', 'links', 'authors']  # Add or modify based on your actual dict-like columns

for column in dict_columns:
    ser_value = df[column].apply(json.dumps)
    df = df.assign(**{column: ser_value})

# Now you can write the DataFrame to PostgreSQL.
df.to_sql("summarize_articles", engine, if_exists='replace')

ProgrammingError: (psycopg2.ProgrammingError) can't adapt type 'dict'
[SQL: INSERT INTO summarize_articles (index, id, link_texts, link_hrefs, title, title_detail, links, link, guidislink, authors, author, author_detail, published, published_parsed, summary, summary_detail, content) VALUES (%(index__0)s, %(id__0)s, %(link_te ... 12881 characters truncated ... published__41)s, %(published_parsed__41)s, %(summary__41)s, %(summary_detail__41)s, %(content__41)s)]
[parameters: {'published_parsed__0': [2024, 1, 7, 19, 58, 22, 6, 7, 0], 'guidislink__0': False, 'link_texts__0': '["National Weather Service", "The Weather Channel"]', 'link__0': 'https://www.sltrib.com/news/2024/01/07/widespread-storm-brings-more-than/', 'id__0': 'https://www.sltrib.com/news/2024/01/07/widespread-storm-brings-more-than/', 'author_detail__0': {'name': 'Kolbie Peterson'}, 'authors__0': '[{"name": "Kolbie Peterson"}]', 'author__0': 'Kolbie Peterson', 'summary__0': '{\'input_documents\': [Document(page_content=\'[{\\

In [18]:
url = os.environ['PG_DB_URL']
engine = create_engine(url)
df.to_sql("summarize_articles", engine, if_exists='replace')
con.close()

ProgrammingError: (psycopg2.ProgrammingError) can't adapt type 'dict'
[SQL: INSERT INTO summarize_articles (index, id, link_texts, link_hrefs, title, title_detail, links, link, guidislink, authors, author, author_detail, published, published_parsed, summary, summary_detail, content) VALUES (%(index__0)s, %(id__0)s, %(link_te ... 12881 characters truncated ... published__41)s, %(published_parsed__41)s, %(summary__41)s, %(summary_detail__41)s, %(content__41)s)]
[parameters: {'published_parsed__0': [2024, 1, 7, 19, 58, 22, 6, 7, 0], 'guidislink__0': False, 'link_texts__0': ['National Weather Service', 'The Weather Channel'], 'link__0': 'https://www.sltrib.com/news/2024/01/07/widespread-storm-brings-more-than/', 'id__0': 'https://www.sltrib.com/news/2024/01/07/widespread-storm-brings-more-than/', 'author_detail__0': {'name': 'Kolbie Peterson'}, 'authors__0': [{'name': 'Kolbie Peterson'}], 'author__0': 'Kolbie Peterson', 'summary__0': '{\'input_documents\': [Document(page_content=\'[{\\\'ty

In [ ]:
def create_author_pairs(row):
    if row and isinstance(row, list):
        return list(itertools.combinations(row, 2))
    return []

# Apply the function to each row and concatenate the results
author_pairs = df['authors'].apply(create_author_pairs).sum()

# Create a DataFrame from the author pairs
author_pairs_df = pd.DataFrame(author_pairs, columns=['Author1', 'Author2'])

# Creating a graph from the DataFrame
G = nx.from_pandas_edgelist(author_pairs_df, 'Author1', 'Author2')

# Draw the network graph
plt.figure(figsize=(12, 8))
nx.draw_networkx(G, with_labels=True, node_color='skyblue', edge_color='gray', font_size=10)
plt.title("Author Collaboration Network")
plt.show()

In [ ]:
df.head()

In [ ]:
# Assuming df is already loaded with the appropriate data

# Function to create pairs of links
def create_link_pairs(row):
    if row and isinstance(row, list):
        return list(itertools.combinations(row, 2))
    return []

# Apply the function to each row and concatenate the results
link_pairs = df['link_hrefs'].apply(create_link_pairs).sum()

# Create a DataFrame from the link pairs
link_pairs_df = pd.DataFrame(link_pairs, columns=['Link1', 'Link2'])

# Count the frequency of each pair
link_pairs_count = link_pairs_df.groupby(['Link1', 'Link2']).size().reset_index(name='Count')


threshold = 2
# Filter out infrequent pairs for clearer visualization
filtered_link_pairs_count = link_pairs_count[link_pairs_count['Count'] > threshold]  # Set a threshold value

# Creating a graph from the DataFrame
G = nx.from_pandas_edgelist(filtered_link_pairs_count, 'Link1', 'Link2', ['Count'])

# Draw the network graph
plt.figure(figsize=(12, 8))
edges = G.edges(data=True)
weights = [edge[2]['Count'] for edge in edges]
nx.draw_networkx(G, with_labels=False, node_size=50, width=weights, edge_color='blue', alpha=0.7)
plt.title("Link Connectivity Network")
plt.show()

In [ ]:
# Filter out infrequent pairs for clearer visualization
threshold = 2  # Set a threshold value for visibility
filtered_link_pairs_count = link_pairs_count[link_pairs_count['Count'] > threshold]

# Creating a graph from the DataFrame
G = nx.from_pandas_edgelist(filtered_link_pairs_count, 'Link1', 'Link2', ['Count'])

# Draw the network graph with labels
plt.figure(figsize=(15, 10))
pos = nx.spring_layout(G, k=0.5)  # k adjusts the distance between nodes
edges = G.edges(data=True)
weights = [edge[2]['Count'] for edge in edges]
nx.draw_networkx(G, pos, with_labels=True, node_size=50, width=weights, edge_color='purple', alpha=0.7, font_size=8)
plt.title("Link Connectivity Network")
plt.axis('off')  # Turn off the axis
plt.show()

In [ ]:
# Assuming df is already loaded with the appropriate data

# Function to create pairs of links
def create_link_pairs(row):
    if row and isinstance(row, list):
        return list(itertools.combinations(row, 2))
    return []

# Function to extract domain from a URL
def get_domain(url):
    try:
        return urlparse(url).netloc
    except:
        return 'Unknown'

# Function to format the path of the URL
def format_domain_label(url):
    path = urlparse(url).path
    # Remove the leading '/' if present
    if path.startswith('/'):
        path = path[1:]
    if len(path) > 48:
        return path[:45] + "..."
    return path
 

# Apply the function to each row and concatenate the results
link_pairs = df['link_hrefs'].apply(create_link_pairs).sum()

# Create a DataFrame from the link pairs
link_pairs_df = pd.DataFrame(link_pairs, columns=['Link1', 'Link2'])

# Extract domains and add them as columns
link_pairs_df['Domain1'] = link_pairs_df['Link1'].apply(get_domain)
link_pairs_df['Domain2'] = link_pairs_df['Link2'].apply(get_domain)

# Count the frequency of each pair
link_pairs_count = link_pairs_df.groupby(['Link1', 'Link2', 'Domain1', 'Domain2']).size().reset_index(name='Count')

# Filter out infrequent pairs for clearer visualization
threshold = 1  # Set a threshold value for visibility
filtered_link_pairs_count = link_pairs_count[link_pairs_count['Count'] > threshold]

# Creating a graph from the DataFrame
G = nx.from_pandas_edgelist(filtered_link_pairs_count, 'Link1', 'Link2', ['Count'])

# Assign a color to each domain
domains = list(set(filtered_link_pairs_count['Domain1']).union(set(filtered_link_pairs_count['Domain2'])))
color_map = {domain: np.random.rand(3,) for domain in domains}

# Draw the network graph with colored nodes
plt.figure(figsize=(15, 10))
pos = nx.spring_layout(G, k=0.5)
edges = G.edges(data=True)
weights = [edge[2]['Count'] for edge in edges]

for node in G:
    nx.draw_networkx_nodes(G, pos, nodelist=[node], node_color=[color_map[get_domain(node)]], node_size=50, alpha=0.7)
nx.draw_networkx_edges(G, pos, width=weights, edge_color='gray', alpha=0.5)
# nx.draw_networkx_labels(G, pos, font_size=8, font_color='black')
# Drawing the labels with formatted domain names
formatted_labels = {node: format_domain_label(node) for node in G.nodes()}
nx.draw_networkx_labels(G, pos, labels=formatted_labels, font_size=8, font_color='black')


# Creating the legend
patch_list = [plt.Line2D([0], [0], marker='o', color='w', label=domain, markersize=10, markerfacecolor=color) for domain, color in color_map.items()]
plt.legend(handles=patch_list, loc='upper right', title='Domain')

plt.title("Link Connectivity Network by Domain")
plt.axis('off')
plt.show()

# plot saving

In [ ]:
plt.figure(figsize=(30, 20))  # You can adjust the size as needed


pos = nx.spring_layout(G, k=0.5)
edges = G.edges(data=True)
weights = [edge[2]['Count'] for edge in edges]

for node in G:
    nx.draw_networkx_nodes(G, pos, nodelist=[node], node_color=[color_map[get_domain(node)]], node_size=50, alpha=0.7)
nx.draw_networkx_edges(G, pos, width=weights, edge_color='gray', alpha=0.5)
# nx.draw_networkx_labels(G, pos, font_size=8, font_color='black')
# Drawing the labels with formatted domain names
formatted_labels = {node: format_domain_label(node) for node in G.nodes()}
nx.draw_networkx_labels(G, pos, labels=formatted_labels, font_size=8, font_color='black')


# Creating the legend
patch_list = [plt.Line2D([0], [0], marker='o', color='w', label=domain, markersize=10, markerfacecolor=color) for domain, color in color_map.items()]
plt.legend(handles=patch_list, loc='upper right', title='Domain')


plt.title("Link Connectivity Network by Domain")
plt.axis('off')

# Save the figure as a PNG file
plt.savefig('network_graph.png', format='png', dpi=300, bbox_inches='tight')

plt.show()
